In [10]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

import urllib

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0        conda-forge
    geopy:           1.20.0-py_0      conda-forge

The following packages will be UPDATED:

    ca-certificates: 2019.5.15-0      anaconda    --> 2019.6.16-hecc5488_0 conda-forge
    certifi:         2019.6.16-py36_0 anaconda    --> 2019.6.16-py36_1     conda-forge

The following packages will be DOWNGRADED:

    openssl:         1.1

In [11]:
page = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

toronto_html = BeautifulSoup(page, 'html.parser')

table_html = toronto_html.body.table.extract()

In [12]:
table = []
for x in table_html.find_all("td"):
    table.append(x.get_text())

table_zip = []
table_borough = []
table_neighborhood = []

for x in range(int(len(table)/3)):
    table_zip.append(table[x*3])

for x in range(int(len(table)/3)):
    table_borough.append(table[x*3+1])
    
for x in range(int(len(table)/3)):
    neighborhood = table[x*3+2]
    table_neighborhood.append(neighborhood[:-1])

In [13]:

column_names = ["PostalCode", "Borough", "Neighborhood"]
neighborhoods = pd.DataFrame(columns = column_names)
neighborhoods

,PostalCode,Borough,Neighborhood


In [14]:
for x in range(int(len(table)/3)):
    neighborhoods = neighborhoods.append({'PostalCode': table_zip[x], 'Borough': table_borough[x], 'Neighborhood': table_neighborhood[x]}, ignore_index = True)
neighborhoods.head(5)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [15]:
neighborhoods = neighborhoods[neighborhoods.Borough != 'Not assigned']
neighborhoods = neighborhoods.reset_index(drop = True)
neighborhoods.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [16]:
for x in range(len(neighborhoods)):
    if neighborhoods.Neighborhood[x] == 'Not assigned':
        neighborhoods.Neighborhood[x] = neighborhoods.Borough[x]

neighborhoods.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [17]:
for x in range(len(neighborhoods)-1):
    if neighborhoods.PostalCode[x] == neighborhoods.PostalCode[x+1]:
        neighborhoods.Neighborhood[x+1] = neighborhoods.Neighborhood[x] + ", " + neighborhoods.Neighborhood[x+1]

In [18]:

length = len(neighborhoods) - 1
neighborhood = neighborhoods
for x in range(length):
    if neighborhoods.PostalCode[x] == neighborhoods.PostalCode[x+1]:
        neighborhood = neighborhood.drop(neighborhoods.index[x])
neighborhood = neighborhood.reset_index(drop = True)
neighborhood.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [19]:

neighborhood.shape

(103, 3)